In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pystac_client import Client
from odc.stac import configure_rio, stac_load
import odc.geo.xr

from remote_sensing_tools.config import de_africa_config
from remote_sensing_tools.data import EOData, LoadInfo

## Set up configuration

In [ ]:
# Open the stac catalogue
catalog = Client.open("https://explorer.digitalearth.africa/stac")

# Set the STAC collections
collections = ["s2_l2a"]

# config 
config = de_africa_config

# Set up connection to South Africa S3
configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
    AWS_S3_ENDPOINT="s3.af-south-1.amazonaws.com",
)

## Set up area of interest and time to load

In [ ]:
# Set a bounding box
# [xmin, ymin, xmax, ymax] in latitude and longitude
bbox = (37.76, 12.49, 37.77, 12.50)

# Set a start and end date
start_date = "2020-09-01"
end_date = "2020-12-01"

# Set geospatial information
input_crs = "EPSG:4326"
input_resolution = 0.001

output_crs = "EPSG:6933"
output_resolution = 20

# Set bands
bands = ("red", "green", "blue", "nir")

In [ ]:
load_info = LoadInfo(bbox=bbox, crs=output_crs, bands=bands, resolution=output_resolution, start_date=start_date, end_date=end_date)

In [ ]:
## Try out class

example = EOData(load_info)

In [ ]:
example.load_info.crs

In [ ]:
example.data.odc.crs

## Search catalogue and load

In [ ]:
# Build a query with the set parameters
query = catalog.search(
    bbox=bbox, collections=collections, datetime=f"{start_date}/{end_date}"
)

# Search the STAC catalog for all items matching the query
items = list(query.items())
print(f"Found: {len(items):d} datasets")

In [ ]:
ds = stac_load(
    items,
    bands=bands,
    crs=output_crs,
    resolution=output_resolution,
    groupby="solar_day",
    chunks={},
    bbox=bbox,
    stac_cfg=config,
)